# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!


In [10]:
# imports
import os

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display


In [ ]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'


In [ ]:
# set up environment

# GPT API KEY
load_dotenv(override=True)
gtp_api_key = os.getenv('OPENAI_API_KEY')
if gtp_api_key and gtp_api_key.startswith('sk-proj-') and len(gtp_api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [5]:
# here is the question; type over this to ask something new
system_prompt = """
You are a helpful assistant that can explain code. Please explain the code in a way that is easy to understand. Add a few examples to help the user understand the code. Respond using markdown.
"""

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [22]:
def stream_response_to_markdown(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

# Get gpt-4o-mini to answer, with streaming

def get_gpt_answer(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt}, {"role": "user", "content": question}
        ],
        stream=True
    )
    return stream_response_to_markdown(stream)


In [ ]:
print(get_gpt_answer(question))

In [24]:
# Get Llama 3.2 to answer

def get_llama_answer(question):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": question}],
        stream=True
    )
    return stream_response_to_markdown(stream)

In [ ]:
print(get_llama_answer(question))